<p align="center">
  <br/>
  <img src="medias/genyus-logo.png" width="150px">
  <br/>
  <br/>
  <h1 align="center">
    <p align="center">
      Owen Gombas<br/>*
      Darmanger David
    </p>
  </h1>
  <h2 align="center">
    <p align="center">
      Juin 2021
    </p>
  </h1>
  <br/>
</p>
<br><br>

# 1. INTRODUCTION

## 1.1 But

C’est dans le cadre de notre cours de python (1242.4 PYTHON) que nous avons effectué ce projet. L’objectif du travail était de prendre en main le langage python et d’avoir une première introduction à l’analyse de données grâce aux outils fournis par le langage. Mais également une prise en main de l’utilisation de notebooks jupyter pour la mise en place du projet.

En ce qui concerne le travail en lui-même, notre projet est un outil d’analyse du “vocabulaire” utilisé dans le rap francophone. Pour être plus précis, on va analyser les paroles d’un échantillon de chansons de rap francophones de ces 20-30 dernières années et les utiliser pour effectuer plusieurs statistiques. Nous avons dans un premier temps délibérément choisi de nous limiter uniquement à des musiques de rap car selon nous, c'est le genre de musique qui jouit de la plus grande diversité de langage et qui a le plus évolué dans le temps durant ces dernières années.

Exemples de statistiques:

    1) Afficher les mots français les plus utilisés, les mots anglais les plus utilisés mais également les mots les plus utilisés du jargon qui ne figure pas officiellement dans le dictionnaire français.
    2) Analyse de quels artistes ont les textes les plus variés en termes de vocabulaire.
    3) Le pourcentage d’utilisation d’un mot particulier sur l’ensemble des chansons ou encore son pourcentage d’utilisation au cours des différentes années. 

## 1.2 API et services utilisés

En ce qui concerne le choix de notre échantillon nous avons effectué une playlist avec un total de 1847 titres différents pour un total de 109h50min de chanson en continue. Nous avons ensuite exporté le nom des musiques et les artistes qui leurs sont associés en un fichier texte, qui sera utilisé pour la récupération des données. Une partie seulement de toutes les chansons seront utilisés en raison de l’API utilisé.
Afin de récupérer les données nécessaires au projet, nous avons besoin de : 


### 1) [Spotify](https://www.spotify.com/fr/)
Grâce Spotify on va facilement pouvoir regrouper les chansons qui nous intéresse.
La playlist utilisée dans le cadre du projet : https://open.spotify.com/playlist/3bp6rWaNssiLhZP431bkU7?si=a5f19a02267a4101


### 2) [Tune My Music](https://www.tunemymusic.com/fr/Spotify-to-File.php)
Ce site permet d'exporter une playlist dans un fichier texte simple au format:
```
Artist - Titre 1
Artist - Titre 2
Artist 2 - Titre 2.2
```
### 3) [Genius](https://genius.com)
Genius est un site fournissant les paroles de musiques, et c'est l'API de ce site que l'on a utilisé afin d'obtenir des informations relative à la musique.

### Fonctionnalités de l'API
Nous avons principalement utilisé que deux fonctionnalités de l'API:
- Recherche globale de musiques, d'artistes ou d'albums
- Récupération des informations d'une musique, d'un artiste ou d'un album

### Récupération des paroles
Malheureusement l'API ne nous fournit pas la possibilité d'obtenir les paroles, on va devoir "scraper" les données avec `beautifulsoup` qui est un module python.

## 1.3 Modules complémentaires
Voici la liste des différents modules utilisés dans le projet :

    - numpy (via anaconda.navigator)
    - Pandas (via anaconda.navigator)
    - Matplotlib (via anaconda.navigator)
	
Utiliser pour l’authentification avec un token à l’API genius

    - Rauth (via terminal: `pip install rauth`)

Scrapping des données :

    - Beautifulsoup4 (via terminal:  `pip install beautifulsoup4`)

Type de graphique spécifique :

    - Wordcloud (via terminal: `pip install wordcloud`)

Pour avoir des composants interactifs:

    - Node.js (via anaconda.navigator)
    - Ipywidgets (via anaconda.navigator)

La liste des modules utilisé dans le projet se trouvent également dans "requirements.txt". Pour les installer, lancez la commande `pip install -r requirements.txt` dans un terminal à la racine du projet au avec un terminal anaconda.

## 1.4 Statistiques principales



Donc le but général du projet est de présenter à l'aide de différentes statistiques l'utilisation la variété des mots utilisés dans les chansons de rap.

# 2. CORPS DU TRAVAIL

## 2.1 Traitement et analyse de la liste des musiques avant récupération
Ici le but est de traiter, nettoyer et analyser les données que l'on a en entré pour notre analyse. Ces données représentent une liste de musiques avec leur artistes (datas/songs.txt)

### Paquets utilisés

In [ ]:
import pandas as pd

### Suppression des doublons
Nous devons supprimer les éventuels doublons qui existent dans cette liste afin d'éviter de fausser nos résultats

In [ ]:
songs: pd.Series

print("Suppression des doublons...", "\n\n")

with open("./datas/songs.txt", "r", encoding="utf8") as file:
    # Read the songs.txt file and create a pandas Serie from it
    songs_inputs = file.read().splitlines()
    songs = pd.Series(songs_inputs)

    print("Avant", "\n", songs, "\n", "...", "\n\n")
    
    # Remove duplicated songs (keep the first)
    songs.drop_duplicates(inplace=True)

    print("Après", "\n", songs, "\n", "...")

    # Rewrite the file without the duplicated values  
    with open("./datas/songs.txt", "w", encoding="utf8") as file:
        file.write("\n".join(songs))
        file.close()

    file.close()

### Représentation des artistes
On va grouper les musiques par artistes, c'est à dire que pour chaque artiste nous faisons correspondre la liste de toutes ses musiques

In [ ]:
# Split each lines using " - " (only the first occurence)
# "Green Montana - Follow-moi" => ["Green Montana", "Follow-moi"]
df_songs = songs.str.split(" - ", expand=True, n=1)
df_songs.columns = ["artist", "song"]

# Group the songs by artists
df_artists_songs = df_songs.groupby("artist")["song"].apply(list).reset_index(name="songs")
df_artists_songs["songs_count"] = df_artists_songs["songs"].apply(lambda x: len(x))
df_artists_songs.sort_values(by="songs_count", inplace=True, ascending=False)

df_artists_songs

### Histogramme des artistes depuis la liste (nettoyées) des musiques 
Nous pouvons maintenant afficher un histogramme depuis la liste (nettoyée) des musiques pour y voir plus clair, on affiche alors chacun de nos artistes avec son nombre de musiques que l'on possède

In [ ]:
df_artists_songs.plot(
    kind="bar",
    x="artist",
    y="songs_count",
    figsize=(20, 10),
    label="Nombre de musiques",
    xlabel="Artiste"
)

## 2.2 Récupération des informations des musiques
Cette partie consiste à la récupération des données via l'API Genius, dans a partie précédente nous avons traité de la partie qui s'occupe des entrées de notre programme. Ici on va justement utiliser cette liste brute de musiques pour récupérer les paroles ainsi que notre information utilise relative à chacune des musiques.  
- Les informations liées à l'artiste
- Les informations liées à l'album où la musique apparait
- Les parole de la musique

### Documentation
La documentation de l'API Genius se trouve [ici](https://docs.genius.com/)

## 2.3 Le scraping de données
Malheureusement Genius ne fournit pas les paroles via son API, on doit alors ruser... La solution est la suivante:  
- On récupère l'URL de la page affichant les paroles via car l'API nous le fournit  
- On fait une requête GET (HTTP) à cette adresse pour récupérer toutes la page HTML affichant les paroles
- Grâce au module BeautifulSoup on extrait uniquement le contenu de la balise HTML contenant les paroles de la musique

### Paquets utilisés

In [ ]:
import json
import requests
import time
import re
import pandas as pd
from datetime import datetime
from typing import List, Any
from rauth import OAuth2Service
from bs4 import BeautifulSoup, PageElement
from IPython.display import clear_output

### Classe pour la manipulation de l'API
Une classe nous simplifie grandement les appels à l'API Genius, nous avons alors fait une classe qui sert à:
- Faire une recherche via l'API
- Récupérer les informations spécifiques à une musique
- Scraper les paroles de la musique

In [ ]:
class API:
    """
    Simplify the requests to the Genius API
    """
    genius: Any
    session: Any

    def __init__(
        self,
        client_id: str,
        client_secret: str,
        authorize_url: str,
        base_url: str,
        token: str,
    ):
        """
        Initialize the Genius API session
        """
        self.genius = OAuth2Service(
            client_id=client_id,
            client_secret=client_secret,
            authorize_url=authorize_url,
            base_url=base_url
        )

        self.session = self.genius.get_session(
            token=token
        )

    def get_lyrics(
        self,
        url: str,
        retry: bool = False,
        wait_retry: int = 30,
        wait: int = 0
    ) -> dict:
        """
        Get the lyrics from the Genius website

        Parameters:
            url (str): The lyrics page URL
            retry (bool): Retry the request if it fails (default False)
            wait_retry (int): Wait x seconds before retrying (default 30)
            wait (int): Wait x seconds before returning the result (default 0)

        Returns:
            lyrics (str or None): The lyrics of the song
        """

        htmlRes = requests.get(url).text

        html = BeautifulSoup(htmlRes, "html.parser")

        # Find the correct lyrics div in the html file.
        # The classname could be "lyrics" or "Lyrics__Root-sc-1ynbvzw-0" 
        # Because the website stack probably use React or Angular with Server Side Rendering
        lyricsDiv: PageElement = html.find("div", class_="lyrics")
        if lyricsDiv == None:
            lyricsDiv = html.find("div", class_="Lyrics__Root-sc-1ynbvzw-0")

        if lyricsDiv != None:
            # We do not want <br>, so we simply replace each occurence with a line return (\n)
            # (could be replaced by a space, because we normalize it just after)
            for br in lyricsDiv.find_all("br"):
                br.replace_with("\n") 

            lyrics: str = lyricsDiv.get_text()

            # We normalize the lyrics by replacing each \n by a simple space (" ")
            # We do not want more than one space between each word so replace chained spaces by a single one ("a    b" => "a b")
            # And finally we remove spaces at the beginning 
            lyrics = re.sub(r"\n", " ", lyrics)
            lyrics = re.sub(r"\s{2,}", " ", lyrics)
            lyrics = lyrics.strip()

            # Optionnaly wait to avoid a ban from Genius's API
            time.sleep(wait)

            return lyrics

        elif retry:
            # Retry if it failed
            print(f"Cannot scrap lyrics... waiting {wait_retry} secondes")
            time.sleep(wait_retry)
            return self.get_lyrics(url, retry, wait_retry, wait)

        return None

    def get_song(
        self,
        id: int,
        with_lyrics: bool = False,
        retry: bool = False,
        wait_retry: int = 30,
        wait: int = 0
    ) -> dict:
        """
        Get a song from the Genius API and parse it into objects (selecting only interesting fields)

        Parameters:
            id (int): The song's ID
            with_lyrics (bool): Get the song with it's lyrics (default False)
            retry (bool): Retry the request if the getting lyrics method failed (default False)
            wait_retry (int): Wait x seconds if the getting lyrics method failed (default 30)
            wait (int): Wait x seconds after the lyrics method (default 0)

        Returns:
            lyrics (dict): The song
        """

        res = self.session.get(f"songs/{id}?text_format=plain").json()
        url = res['response']['song']['url']
        lyrics = ""

        album: dict = None

        # Extract only interesting informations about the song
        if res['response']['song']['album']:
            album = {
                "name": res['response']['song']['album']['name'],
                "id": res['response']['song']['album']['id']
            }            

        artist = {
            "name": res['response']['song']['primary_artist']['name'],
            "id": res['response']['song']['primary_artist']['id'],
            "url": res['response']['song']['primary_artist']['url'],
            "image": res['response']['song']['primary_artist']['image_url']
        }

        song = {
            "id": res['response']['song']['id'],
            "name": res['response']['song']['title'],
            "album": album,
            "artist": artist,
            "image": res['response']['song']['header_image_url'],
            "url": url,
            "original_lyrics": lyrics,
            "date": datetime.strptime(
                res['response']['song']['release_date'] or "1900-01-01", "%Y-%m-%d").isoformat()
        }

        if with_lyrics:
            lyrics = self.get_lyrics(url, retry, wait_retry, wait)

        if lyrics != None:
            song["original_lyrics"] = lyrics

        return song

    def search(self, query: str) -> List[dict]:
        """
        Search for an album, artist, song in the Genius API

        Parameters:
            query (str): The search query

        Returns:
            Results (List[dict]): The provided results
        """

        search = self.session.get(f"search?q={query}").json()
        hits = search['response']['hits']
        results: List[dict] = []

        for hit in hits:
        # Extract only interesting informations about the searched song
            hitResult = hit['result']

            artist = {
                "name": hitResult['primary_artist']['name'],
                "id": hitResult['primary_artist']['id'],
                "image": hitResult['primary_artist']['image_url'],
                "url": hitResult['primary_artist']['url']
            }

            search = {
                "title": hitResult['title'],
                "url": hitResult['url'],
                "artist": artist,
                "id": hitResult['id']
            }

            results.append(search)

        return results

    def print_json(content: str):
        print(json.dumps(content, indent=2, sort_keys=True))

### Authentification à l'API
Afin de faire des requêtes à l'API nous devons nous authentifier avec un token OAuth  
Ce token peut être généré à l'adresse suivante: [https://genius.com/api-clients](https://genius.com/api-clients)

In [ ]:
# For the demo we let the tokens on purpose but we should avoid doing that
api = API(
    client_id="xAP0jvOkLrC3eAjwE4iCeY5BdSrgH7qKUQyh8907-2fGiAGYEHJMNhtFglSLznAq",
    client_secret="WIVq7t1Jq5uaN0OkYCPzhVMr4mt_d-ufoq5fSC6qmyUaxodx5kZ4bS56J87C-LXGRqeeXp9nFpjgrgPtZ_8niA",
    authorize_url="https://api.genius.com/oauth/authorize",
    base_url="https://api.genius.com/",
    token="qfhBonIalyiGK0DcsHmg3-heXf485c1dSV-gOM3ZU4Wn3eD-6-pKjESnhYg4kJ1y"
)

### Récupération d'une musique avec l'API Genius

In [ ]:
song = api.get_song(1)
API.print_json(song)

### Recherche d'une musique avec l'API Genius

In [ ]:
results = api.search("damso")
API.print_json(results)

### Récupération des paroles d'une musique

In [ ]:
lyrics = api.get_lyrics(song["url"])
lyrics

## 2.4 Scraping des données
Récupération de toutes les musiques d'un à partir du fichier contenant les musiques (datas/songs.txt)  
**Le code n'est pas exécuté (la fonction `scrap` n'est pas appelée, car commentée), cette opération peut durer plusieurs heures. Nous avons déjà effectué ce scraping afin de nourrir notre fichier songs.json, qui représente entre-autre notre base de données des musiques**

Comme nous l'avons déjà détaillé, les opérations suivantes pour chacune des musiques sont effectuées pour scraper les paroles

- On utilise la méthode de recherche de la classe API (`api.search`)

- Si des résultats sont donnés, on opère uniquement sur le premier résultat (qui est probablement celui qui nous intéresse) on récupéré les informations de ce résultat ("hit") qui représente la musique pas dans sa totalité, c'est la raison pour laquelle on fait cette opération  (`api.get_song`)  
Cette opération va également récupérer les paroles de la musique directement (grâce aux argument passés à la méthode)

- On ajoute cete musique dans une liste de résultats et on exporte cette liste en JSON dans le fichier `api_songs.json`

> On exporte le fichier à chaque itération car si la boucle est interrompue par une erreur on peut quand même récupérer la liste des musiques qui ont été récupérées

In [ ]:
input_path: str = "./datas/songs.txt"
output_path: str = "./datas/api_songs.json"

def scrap():
    with open(input_path, "r", encoding="utf8") as file:
        songs_inputs = file.read().splitlines()
        songs: List[str] = []
      
        for index, song_input in enumerate(songs_inputs, start=1):
            # Search for a song with a query for example: "Green Montana - CALIFORNIA"
            results = api.search(song_input)

            # Retrieve all the songs informations and lyrics of the first result
            # (the first result is the one that we are interested in)
            if len(results) > 0:
                song = api.get_song(results[0]["id"], True, True, 0, 0)
                songs.append(song)

                print(f"{index / len(songs_inputs) * 100:.2f}%     {song['artist']['name']} - {song['name']}")
            
            # Write the file in JSON format at api_songs.json
            with open(output_path, "w", encoding="utf8") as output:
                output.write(
                    json.dumps(
                        songs,
                        default=lambda obj: obj.__dict__,
                        ensure_ascii=False
                    )
                )
                output.close()
        file.close()

    print("Le fichier JSON contenant les musiques a été écrit dans: ", output_path)

# We avoid calling this function because it can easily last few hours
# The result of our songs list is already in the file api_songs.json
# scrap()

## 2.5 Clean des sons après récupération
On ne peut pas se fier complètement à la recherche de l'API Genius, pour deux raisons:

- Le son peut potentiellement ne pas exister dans la base de données de Genius

- Le premier résultat n'est pas forcément le bon, on proposition d'amélioration aurait pu d'être de parcourir tous les résultats d'une recherche et vérifier si la musique recherchée n'apparait pas comme le deuxième, troisième, ..., résultat

Pour ces raisons nous allons filtrer que les musique qui étaient bel et bien dans notre liste de musique au départ

### Paquets utilisés

In [ ]:
import json
import re
import pandas as pd
from typing import List

### Application du tri
En entrée on va prendre le fichier comportant la liste des musiques que l'on a scrapé auparavant (`api_songs.json`) et on va en premier temps vérifier, pour chacune des musiques, qu'elle existant au départ dans notre liste de sons brute (`songs.txt`) en recréant un champ `Artiste - Titre de musique` à partir des informations réel de la musique. En deuxième temps, nous allons simplement supprimer tous les potentiels doublons qui pourrait apparaitre dans notre liste. Pour finalement afficher le pourcentage de musique que l'on peut exploiter.

In [ ]:
input_path = "./datas/api_songs.json"
output_path = "./datas/songs.json"

songs = pd.read_json(input_path)
inputs: pd.Series

with open("./datas/songs.txt", "r", encoding="utf8") as file:
    inputs = pd.Series(file.read().lower().splitlines())
    file.close()

# We recreate the field "Artist - Title" in the DataFrame
songs["artist_name"] = songs['artist'].apply(pd.Series)["name"]
songs["query"] = songs['artist_name'].str.lower() + " - " + songs['name'].str.lower()

# We verify that the song exists in the file songs.txt
songs["is_in_inputs"] = songs["query"].isin(inputs)

# We filter the songs and remove the songs that aren't the the file songs.txt
songs = songs[songs["is_in_inputs"] == True]

# We remove the duplicated songs
songs.drop_duplicates(subset=["id"], inplace=True)

del songs["is_in_inputs"]
del songs["artist_name"]

print(f"Avec {len(inputs)} musiques, Genius a fournit {len(songs)} musiques de façon correcte")
print(f"Nous gardons alors {len(songs) / len(inputs) * 100:.2f}% de nos musiques pour la suite")

songs

## 2.6 Création de plusieurs champs dérivé des paroles originales
Nous allons avoir besoin de plusieurs variations des paroles afin de procéder à nos statistiques.  
Notamment pour analyser grâce un dictionnaire de mots d'une langue les mots existant dans cette langue (`lyrics_keywords`)

### `original_lyrics`
Les paroles originales, celles récupérées sur le site [genius.com](https://genius.com)

### `lyrics`
Les paroles sans: `[Couplet]`, `mot94`, `34`, `  `
- Supprime les accolades et leur contenu "\[...]"
- Supprime toutes les parenthèses et leur contenu
- Supprime tous ce qui n'est pas une lettre, une apostrophe ou un tirait (a-Z, ', -)
- Supprime les lettres seules "abc d efg" => "abc efg"
- Remplace plusieurs espaces par un seul "abc   def" => "abc def"

### `lyrics_keywords`
Les paroles utilisées pour être comparée avec un dictionnaire, elles sont normalisées à leur forme la plus simple, effectue le même nettoyage que pour le champ `lyrics` mais sans les mot basé sur des apostrophes, les apostrophes, les tirait, etc...

In [ ]:
# 1. Remove: "[...]"
# 2. Remove all the brackets and their contents "(...)"
# 3. Remove everything that is not a letter (a-Z) a word letter a single quote "'" or a "-"
# 4. Remove alone letters "abc d efg" => "abc efg"
# 5. Replace chained spaces by a single one "abc   def" => "abc def"
def clean_lyrics(lyrics: str):
    lyrics = re.sub(r"\([^()]*\)", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"\s*\[(.*?)\]\s*", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"[^a-zA-Z \w ' -]", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"\w*\d\w*", "", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"\s{2,}", " ", lyrics, flags=re.M | re.I)
    lyrics = lyrics.strip().lower()

    return lyrics

# Clean the lyrics like clean_lyrics does
# but it remove also the letters/words that can't be compared to a dictionnary
def clean_lyrics_keywords(lyrics: str):
    lyrics = clean_lyrics(lyrics)
    lyrics = re.sub(r"(^|\s)(qu'|j'|l'|t'|c'|t'|d'|s'|n'|y'|m')*", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"'(\s|$)", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"(\s|^)'", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"-(\s|$)", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"(\s|^)-", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"(^| ).(( ).)*( |$)", " ", lyrics, flags=re.M | re.I)
    lyrics = re.sub(r"\s{2,}", " ", lyrics, flags=re.M | re.I)
    lyrics = lyrics.strip().lower()

    return lyrics

songs["lyrics"] = songs["original_lyrics"].apply(clean_lyrics)
songs["lyrics_keywords"] = songs["original_lyrics"].apply(clean_lyrics_keywords)

songs_json = songs.to_json(force_ascii=False, orient="records")

with open(output_path, "w", encoding="utf8") as file:
    file.write(songs_json)
    file.close()

print("Le fichier JSON nettoyé a été écrit dans: ", output_path)

## 2.7 Analyse des mots avec des dictionnaires de langues
Nous passons à la partie des statistiques, on va afficher les mots créer dans le rap qui ne figure pas dans le dictionnaire français, ceci a pour but de faire ressortir les mots du verlan ainsi que les mots créer par ce genre musical (argots, créations, mot d'autres origines, etc.…). Nous allons également analyser les anglicismes les plus utilisés pour finir par afficher les mots français étant les plus employés.

### Paquets utilisés
On aura besoin des librairies externe: 
- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [matplotlib](https://matplotlib.org/)
- [wordcloud](https://amueller.github.io/word_cloud/)

In [ ]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as ipw
from wordcloud import WordCloud
from typing import List

### Ouverture des fichiers contenant les données à utiliser

In [ ]:
songs: pd.DataFrame
french: pd.Series
english: pd.Series

songs = pd.read_json("./datas/songs.json")

with open("./datas/french.txt", "r", encoding="utf8") as file:
        french = pd.Series(file.read().lower().splitlines())
        file.close()

# Generated with http://app.aspell.net/create
with open("./datas/english.txt", "r", encoding="utf8") as file:
        english = pd.Series(file.read().lower().splitlines())
        file.close()

### Sélection et manipulation des données afin de pouvoir les utiliser
On va simplement comparer tous les mots que l'on possède avec le champ `lyrics_keywords`, compter leur nombre d'apparition et filtrer cette liste de musiques en vérifiant que ces mots ne figurent pas dans le dictionnaire français et anglais car on ne veut pas afficher les anglicismes ici

In [ ]:
# Group all the words of all the songs in a single pd.Serie
words: pd.Series = songs["lyrics_keywords"].apply(lambda x: x.split(" ")).explode()

# Number of every differents words in all lyrics
all_words_count=len(words)

# Count all the words in this array
occurences: pd.Series = words.value_counts()

# Create a Dataframe that contains the number of occurence
# and if the word is inside the french dictionary
data = {
    "count": occurences,
    "is_french": occurences.index.isin(french),
    "is_english": occurences.index.isin(english)
}

description = pd.DataFrame(
    data=data,
    columns=["count", "is_french", "is_english"],
    index=occurences.index
)

#Select all the words and their count
words_count = pd.Series(description["count"], description.index)

# Select only the words that aren't french
not_french = description[description["is_french"] == False]
not_french = not_french[not_french["is_english"] == False]

frequency = pd.Series(not_french["count"], index=not_french.index)

with open("./datas/not_french.csv", "w", encoding="utf8") as f:
        f.write(frequency.to_csv())
        f.close()

not_french

Ceci est la liste des mots de figurant pas dans le dictionnaire français avec leurs occurrences (globale, parmi toutes les musiques)

# 3. STATISTIQUES SUR L'UTILISATION DES MOTS

## 3.1 Recherche d'un mot particulier et pourcentage
Pour jouer un peu avec la liste créée ci-dessus vous pouvez entrer un mot dans le champ ci-dessus afin de voir son nombre d'occurrences dans les paroles

In [ ]:
# Textbox creation
word_textbox = ipw.Text(value="", placeholder="Search word", description="Word to count:", disabled=False)

# Function called when the button is clicked
def textbox_changed(word: str):
    word_searched = word.lower().strip()

    if word_searched in words_count:
        #number of times used
        print(f"Le mot \"{word_searched}\" est utilisé {words_count[word_searched]} fois dans toutes les paroles\n")
        #percentage of use
        percentage_used_word = round(words_count[word_searched] / all_words_count * 100, 2)
        print(f"Pourcentage d'utilisation: {percentage_used_word}%")
    else:
        print(f"Le mot \"{word_searched}\" ne se trouve pas dans les paroles")

# Button interaction
ipw.interact_manual(textbox_changed, word=word_textbox.value)

## 3.2 Les mots ne figurant pas dans le dictionnaire français (ni anglais)
### L'argot, le ver-lan, etc...

### WordCloud, plus le mot est grand, plus il est utilisé
Voici une représentation des 50 mots en "word-cloud" les plus utilisés ne figurant pas dans le dictionnaire français

Pour cette représentation on a utilisé le module [WordCloud](https://amueller.github.io/word_cloud/)

In [ ]:
wordcloud = WordCloud(
    font_path="./medias/poppins.ttf",
    background_color="white",
    width=1600,
    height=800,
    max_words=50
).generate_from_frequencies(frequency.to_dict())

plt.figure(figsize=(14, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Histogramme des mots
Une autre représentation possible de ces mots est en histogramme, voici alors les 30 mots les plus utilisés avec leurs nombres d'occurrence

In [ ]:
frequency.head(30).plot(
    kind="bar",
    figsize=(14, 7),
    ylabel="Nombre d'utilisation",
    xlabel="Mot"
)

## 3.3 Les anglicismes
On va analyser les mots les plus utilisés provenant de l'anglais

In [ ]:
english_words = description[description["is_french"] == False]
english_words = english_words[english_words["is_english"] == True]

frequency = pd.Series(english_words["count"], index=english_words.index)

with open("./datas/english_words.csv", "w", encoding="utf8") as f:
        f.write(frequency.to_csv())
        f.close()

english_words

### WordCloud, plus le mot est grand, plus il est utilisé
Voici une représentation des 50 mots en "word-cloud" les plus utilisés figurant dans le dictionnaire anglais

Pour cette représentation on a utilisé le module [WordCloud](https://amueller.github.io/word_cloud/)

In [ ]:
wordcloud = WordCloud(
    font_path="./medias/poppins.ttf",
    background_color="white",
    width=1600,
    height=800,
    max_words=50
).generate_from_frequencies(frequency.to_dict())

plt.figure(figsize=(14, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Histogramme des mots
Une autre représentation possible de ces mots est en histogramme, voici alors les 30 mots les plus utilisés avec leurs nombres d'occurrence

In [ ]:
frequency.head(30).plot(
    kind="bar",
    figsize=(14, 7),
    ylabel="Nombre d'utilisation",
    xlabel="Mot"
)

## 3.4 Le "vrai" français
On va afficher les mots les plus utilisés figurant uniquement dans le dictionnaire français

In [ ]:
french_words = description[description["is_french"] == True]

frequency = pd.Series(french_words["count"], index=french_words.index)

with open("./datas/french_words.csv", "w", encoding="utf8") as f:
        f.write(frequency.to_csv())
        f.close()

french_words

### WordCloud, plus le mot est grand, plus il est utilisé
Voici une représentation en "word-cloud" des 50 mots les plus utilisés figurant dans le dictionnaire français

Pour cette représentation on a utilisé le module [WordCloud](https://amueller.github.io/word_cloud/)

In [ ]:
wordcloud = WordCloud(
    font_path="./medias/poppins.ttf",
    background_color="white",
    width=1600,
    height=800,
    max_words=50
).generate_from_frequencies(frequency.to_dict())

plt.figure(figsize=(14, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Histogramme
Une autre représentation possible de ces mots est en histogramme, voici alors les 30 mots les plus utilisés avec leurs nombres d'occurrence

In [ ]:
frequency.head(30).plot(
    kind="bar",
    figsize=(14, 7),
    ylabel="Nombre d'utilisation",
    xlabel="Mot"
)

## 3.5 Répartition du language
Finalement, il est intéressant d'afficher grâce à un pie chart", la répartition de ces langues/langages

In [ ]:
# Count number of words by language
french_words_count = len(description[description["is_french"] == True])
english_words_count = len(english_words)
other_words_count = len(not_french)

datas = [french_words_count, english_words_count, other_words_count]
labels = ["French", "English", "Other (slang)"]

plt.figure(figsize=(14, 7))
plt.title("Usage of language")

plt.pie(datas, labels=labels, autopct='% 1.1f%%')
plt.show()

# 4. STATISTIQUES SUR LES ARTISTES
Pour le moment on n'a fait que des traitements sur les musiques sans prendre en compte l'artiste, ici on va justement traiter de ceux-ci.

La difficulté est alors de "retourner la chaussette", c'est à dire de regrouper toutes les musiques par leur artiste ou lieu d'avoir une grande liste de musiques non groupée

### Paquets utilisés

In [ ]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as ipw
from wordcloud import WordCloud
from typing import List

## 4.1 Regroupement des musiques par artistes
Cette opération est plutôt fastidieuse, on va premièrement créer un champ contenant toutes les informations de l'artiste (dictionnaire) qui va nous permettre de grouper les musiques (par ce champs)

In [ ]:
songs = pd.read_json("./datas/songs.json")

songs["lyrics_list"] = songs["lyrics"].apply(lambda x: x.split(" "))

artist_as_dataframe = songs['artist'].apply(pd.Series)
artist_as_dataframe.rename(columns=lambda x: "artist_" + str(x), inplace=True)

album_as_dataframe = songs['album'].apply(pd.Series)
album_as_dataframe.rename(columns=lambda x: "album_" + str(x), inplace=True)

songs = pd.concat([
    artist_as_dataframe,
    album_as_dataframe,
    songs
], axis=1)

songs["artist_dict"] = songs["artist"].apply(lambda x: json.dumps(x))

songs

In [ ]:
artists_songs = pd.DataFrame(songs.groupby("artist_dict").apply(lambda g: g.to_dict("records"))).reset_index()

artists_songs.columns = ["artist", "songs"]

artists_songs["artist"] = artists_songs["artist"].apply(lambda x: json.loads(x))

artists = pd.DataFrame.from_dict(artists_songs["artist"].to_dict(), orient="index")

artists["songs"] = artists_songs["songs"]
artists["songs_count"] = artists_songs["songs"].map(lambda x: len(x))

artists = artists.sort_values("songs_count", ascending=False)

artists["artist_id"] = artists["name"].str.lower()
artists.set_index("artist_id", inplace=True)

artists

On obtient ici la liste de tous les artistes avec un champs contenant la liste de ses musiques sous forme de dictionnaire

## 4.2 Représentation des pertes du au nettoyage des musiques au niveau des artistes
Grâce à ce groupement par artiste on va pouvoir représenter graphiquement les pertes engendrées par le nettoyage du fichier `api_songs.json`

La barre bleue montre le nombre de musiques que l'on voulait récupérer de base, en orange c'est ce que l'on a pu obtenir après le nettoyage des données

In [ ]:
songs_inputs: pd.Series

with open("./datas/songs.txt", "r", encoding="utf8") as file:
    songs_inputs = pd.Series(file.read().splitlines())
    file.close()

songs = songs_inputs.str.split(" - ", expand=True, n=1)
songs.columns = ["artist", "song"]

# Group the songs by artists
artists_comparaison = songs.groupby("artist")["song"].apply(list).reset_index(name="songs")
artists_comparaison["songs_count"] = artists_comparaison["songs"].apply(lambda x: len(x))
artists_comparaison.sort_values(by="songs_count", inplace=True, ascending=False)

artists_comparaison["artist_id"] = artists_comparaison["artist"].str.lower()
artists_comparaison.set_index("artist_id", inplace=True)

artists_comparaison["songs_count_after_clean"] = artists["songs_count"]
artists_comparaison = artists_comparaison[["artist", "songs_count", "songs_count_after_clean"]]

# Display the histogram
artists_comparaison.columns = ["Artiste", "Nombre de musiques initial", "Nombre de musiques après filtre"]
artists_comparaison.plot.bar(
    x="Artiste",
    figsize=(25, 10)
)

## 4.3 Calcul du pourcentage de répétition
Une statistique plutôt intéressante est de mesurer la richesse du vocabulaire des artistes en montrant lesquels se répètent le plus dans leurs paroles via un histogramme

In [ ]:
def select_words(song: dict):
    songs_count = len(song)

    song_df = pd.DataFrame(song)
    words = song_df["lyrics"].map(lambda x: x.split(" ")).explode().to_list()

    return words

def select_unique_words(words: list):
    return pd.Series(words).drop_duplicates().to_list()

# All the used words of the artist
artists["words"] = artists["songs"].map(select_words)
artists["words_count"] = artists["words"].map(lambda x: len(x))

# All the used words os the artist but without duplications
artists["unique_words"] = artists["words"].map(select_unique_words)
artists["unique_words_count"] = artists["unique_words"].map(lambda x: len(x))

# Count the number of repetition
artists["reptition_count"] = artists["words_count"] - artists["unique_words_count"]

# Make a percentage of the repetitions
artists["repetition_percentage"] = artists["reptition_count"] / artists["words_count"] * 100

artists.sort_values(by="repetition_percentage", inplace=True)

artists

## 4.4 Classement des artistes se répétant du moins au plus

In [ ]:
artists.plot(
    kind="bar",
    x="name",
    y="repetition_percentage",
    figsize=(20, 10),
    xlabel="Artiste",
    label="Pourcentage de répétition"
)

# 5. STATISTIQUES DANS LE TEMPS
Nous récupérons également une date sur chacune des musiques, il est alors intéressant d'exploiter cette donnée afin de relever l'évolution du langage dans le temps.

Comme représentation nous avons choisi de faire deux histogrammes:
- Un premier avec comme axe x les années et en axe y la moyenne du pourcentage d'utilisation du mot **par musique** durant l'année.

- Un deuxième avec comme axe x les années et en axe y le nombre de musiques utilisant ce mot durant l'année

Cette façon de représenter cette donnée statistique peut toujours être contestée, il y'a effectivement plusieurs manières de faire ressortir cette donnée. Mais celle-ci à l'avantage d'être cohérente. Prenons pour exemple le mot "le", statistiquement il est juste de se dire que l'utilisation de ce mot ne varie pas d'années en années car c'est un mot que l'on peut qualifier d'indispensable pour composer un texte. Et notre méthode garantie cette cohérence. Ainsi, si un artiste utilise un mot beaucoup de fois sur une musique cela peut nous induire en erreur par exemple.

Initialement notre réprésentation était plutôt incomplète, en axe y on représentais le pourcentage d'utilisation du mot durant toute l'année sans prendre en compte son appartenance à une certaine musique. Ce qui peut causer des erreurs d'interpétation du graphique.

Cette statistique, est très intéressantes en dehors de l'aspect mathématique de la chose. On peut étudier, grâce à ça: l'évolution des mentalités, analyser les tendances / modes, ainsi que de tracker la création de certains mots.

### Paquets utilisés

In [ ]:
import numpy as np
import json
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as ipw
from wordcloud import WordCloud
from typing import List

## 5.1 Regroupement des mots par années

In [ ]:
songs = pd.read_json("./datas/songs.json")

songs["year"] = songs["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S").year)

songs.sort_values("year", ascending=False, inplace=True)

songs["words"] = songs["lyrics_keywords"].map(lambda x: x.split(" "))

songs = songs.groupby("year")["lyrics_keywords"].apply(list)
songs = songs.reset_index()

def split_keyword(keywords_list: List[str]):
    return [keywords.split(" ") for keywords in keywords_list]
songs["lyrics_keywords"] = songs["lyrics_keywords"].apply(split_keyword)

songs["songs_count"] = songs["lyrics_keywords"].map(lambda x: len(x))

songs = songs[songs["year"] > 1900]

songs

## 5.2 Représentation du nombre de nos musiques par année

In [ ]:
songs.plot(
    kind="bar",
    x="year",
    y="songs_count",
    figsize=(14, 7),
    xlabel="Année",
    label="Nombre de musiques"
)

## 5.2 Affichages de l'utilisation d'un mot par années
Ici vous pouvez simplement taper un mot afin d'afficher les deux histogrammes mentionnés au début du chapitre.

In [ ]:
# Textbox creation
word_textbox = ipw.Text(value="", placeholder="Search word", description="Word to count:", disabled=False)

songs_count: pd.DataFrame
searched_word: str

# Here we calculate, for each songs, the percentage of usage of the word
def percentage_usage_per_song(word: str, words: List[List[str]]):
    words_list = pd.Series(words)

    def count_percentage(words: List[str]):
        words = pd.Series(words)
        return len(words[words == word]) / len(words) * 100

    percentage_usage_per_song = words_list.map(count_percentage)

    return percentage_usage_per_song

# We want to count how many songs use the word
def count_songs(percentage_usage_per_song: pd.Series):
    return len(percentage_usage_per_song[percentage_usage_per_song > 0])

# Function called when the button is clicked
def textbox_changed(word: str):
    searched_word = word
    word_searched = word.lower().strip()

    songs_count = songs

    songs_count["percentage_usage_per_song"] = songs_count["lyrics_keywords"].map(lambda x: percentage_usage_per_song(word, x))

    # We want to calculate the average of the percentage of usage for each years
    songs_count["average_usage"] = songs_count["percentage_usage_per_song"].map(lambda x: x.mean())
    songs_count["number_of_songs"] = songs_count["percentage_usage_per_song"].map(count_songs)

    fig, axes = plt.subplots(nrows=1, ncols=2)
    
    songs_count.plot.bar(
        x="year",
        y="number_of_songs",
        figsize=(25, 5),
        ax=axes[0],
        xlabel="Année",
        label="Nombre de musiques utilisant le mot"
    )

    songs_count.plot.bar(
        x="year",
        y="average_usage",
        figsize=(25, 5),
        ax=axes[1],
        color=(1, 0, 0),
        xlabel="Année",
        label="Moyenne du pourcentage d'utilisation du mot"
    )

# Button interaction
ipw.interact_manual(textbox_changed, word=word_textbox.value)

# 6. LIMITATIONS ET PERSPECTIVES

La limitation principale de notre projet est évidemment sur l’échantillons de données que nous avons. Pour ce cas d’école, nous avons tout de même une bonne quantité de données pour les statistiques que nous effectuons. Néanmoins en augmentant encore le nombre de chansons de différents artistes de différentes années, les résultats pourraient être encore plus précis. 
De plus, comme dis précédemment, si on prend un exemple précis de l’affichage de l’utilisation d’un mot par années. Si un artiste utilise un mot un grand nombre de fois cela peut parfois fausser la statistique. 

L'amélioration la plus intéressante qui pourrait être faite est celle de comparer différents genres musicaux. Comme par exemple, comparer le pourcentage d’utilisation d’un mot particulier dans le rap, rock et la pop. On pourrait également comparer la variété du vocabulaire de différents artistes des différents genres pour voir dans quel style il y a le moins de répétitions de mots. 
Grâce à notre façon de calculer cette statistique, il nous est aisé de d'appliquer cette amélioration. Il suffirait d’ajouter des chansons de différents styles musicales dans le fichier qui contient tous les titres des musiques et qu’il soit aussi disponible sur le site de Genius.

Une autre amélioration envisageable serait de pouvoir ajouter directement une chanson qui sera utilisé dans les statistiques grâce à une interface intégré au notebook. Cela augmentera beaucoup l’utilisation de notre projet pour des utilisateurs moins expérimentés.

Malheureusement nous n’avons pas pu réaliser ces fonctionnalités en raison d'un manque de temps. 

# 7. CONCLUSION

En dehors de la problématique de la récupération des paroles de chansons, nous n'avons pas particulièrement rencontré de trop grosses difficultés lors de la réalisation de ce projet. 
Il nous a permis de nous familiariser avec la « transformation » de données pour les avoir dans le format souhaité. En plus de cela, on a pu avoir une première approche des outils que fournissent le langage python pour réaliser des graphiques. 

Nous avons pu terminer le travail dans les temps. Toutes les fonctionnalités prévues initialement ont pu être réalisé et quelques ajouts ont également été effectué. Nous aurions aimé pouvoir achever complétement le projet avant de le rendre avec les quelques améliorations citées au-dessus mais nous sommes déjà satisfaits du résultat actuel. 

Pour conclure, nous espérons que nous avons pu montrer avec genyus et ses différentes statistiques, la variété et même l’évolution du vocabulaire utilisé dans le rap francophone.


<p align="center">
  <br/>
  <img src="medias/he-arc-logo.png" width="450px">
  <br/>
  <br/>
  <p align="center">Haute-École Arc (Tous droits réservés)
  </p>
  <br/>
</p>